```{contents}
```
## Local Runtime

The **Local Runtime** in LangGraph is the **execution environment** that runs a compiled graph **on the developer’s own machine or server process** rather than in a managed cloud service.
It is the default and most important runtime mode for **development, debugging, experimentation, and many production deployments**.

---

### **1. What “Runtime” Means in LangGraph**

A **runtime** is the system responsible for:

| Responsibility   | Description                       |
| ---------------- | --------------------------------- |
| Graph execution  | Scheduling nodes and edges        |
| State management | Storing and updating shared state |
| Concurrency      | Handling async & parallel nodes   |
| Control flow     | Enforcing routing & termination   |
| Fault handling   | Retries, timeouts, failures       |
| Observability    | Logging, tracing, debugging       |

**Local Runtime = all of this runs inside your Python process.**

---

### **2. Architecture of Local Runtime**

```
Your Python Process
┌─────────────────────────────────────────┐
│ LangGraph Execution Engine              │
│  ├─ Graph Scheduler                     │
│  ├─ Node Executors                      │
│  ├─ State Manager                       │
│  ├─ Checkpoint Manager                  │
│  ├─ Router / Control Flow Engine        │
│  ├─ Error & Retry Controller            │
│  └─ Observability Hooks                 │
└─────────────────────────────────────────┘
```

Everything runs **in-memory** (unless you attach persistence).

---

### **3. When Local Runtime Is Used**

| Scenario               | Why                     |
| ---------------------- | ----------------------- |
| Development            | Fast iteration          |
| Debugging              | Step-by-step inspection |
| Research               | Controlled experiments  |
| Edge deployments       | No cloud dependency     |
| Small-scale production | Simplicity & control    |

---

### **4. Execution Lifecycle**

1. **Graph Definition**
2. **Compilation**
3. **Invocation**
4. **Node Scheduling**
5. **State Mutation**
6. **Edge Resolution**
7. **Checkpointing**
8. **Termination**

```python
graph = builder.compile()
result = graph.invoke(input_state)
```

---

### **5. State Behavior in Local Runtime**

| Feature     | Behavior                          |
| ----------- | --------------------------------- |
| Storage     | In-memory by default              |
| Mutability  | Controlled via reducers           |
| Persistence | Optional via checkpointer         |
| Recovery    | Supported if persistence attached |
| Concurrency | Thread / async based              |

Example:

```python
from langgraph.checkpoint.sqlite import SqliteSaver
checkpointer = SqliteSaver("state.db")
graph = builder.compile(checkpointer=checkpointer)
```

---

### **6. Concurrency Model**

Local runtime supports:

* Synchronous execution
* Async nodes
* Parallel fan-out / fan-in
* Cooperative scheduling

```python
result = await graph.ainvoke(state)
```

---

### **7. Fault Handling & Safety**

| Mechanism        | Purpose                       |
| ---------------- | ----------------------------- |
| Retries          | Recover from transient errors |
| Timeouts         | Prevent hangs                 |
| Recursion limits | Prevent infinite loops        |
| Checkpointing    | Resume after crash            |
| Interrupts       | Human-in-the-loop             |

```python
graph.invoke(state, config={"recursion_limit": 25})
```

---

### **8. Observability in Local Runtime**

Local runtime provides full introspection:

* Execution traces
* State snapshots
* Node timings
* Failure logs

Works natively with **LangSmith** for visualization and replay.

---

### **9. Production Use of Local Runtime**

Local runtime is production-grade when combined with:

* External state store
* External memory store
* Containerization (Docker)
* Process supervision (systemd, k8s)
* Monitoring & alerting

---

### **10. Local vs Cloud Runtime**

| Feature   | Local Runtime | Managed Runtime |
| --------- | ------------- | --------------- |
| Control   | Full          | Limited         |
| Latency   | Minimal       | Higher          |
| Cost      | Low           | Variable        |
| Scaling   | Manual        | Automatic       |
| Debugging | Excellent     | Limited         |

---

### **11. Mental Model**

Local Runtime behaves like an **operating system for your LLM application** running inside Python:

> **Graph = Program**
> **State = Memory**
> **Nodes = Instructions**
> **Edges = Control Flow**



### Demonstration

In [4]:
# Local Runtime Demonstration (Single Cell)

from typing import TypedDict
from langgraph.graph import StateGraph, END
from langgraph.checkpoint.sqlite import SqliteSaver
import sqlite3

# ---- 1. Define State Schema ----
class State(TypedDict):
    count: int

# ---- 2. Define Nodes ----
def increment(state: State):
    print("Increment:", state["count"])
    return {"count": state["count"] + 1}

def router(state: State):
    if state["count"] >= 5:
        return END
    return "increment"

# ---- 3. Build Graph ----
builder = StateGraph(State)
builder.add_node("increment", increment)
builder.set_entry_point("increment")

builder.add_conditional_edges("increment", router, {
    "increment": "increment",
    END: END
})

# ---- 4. Attach Persistence (Local Runtime Feature) ----
checkpointer = SqliteSaver(sqlite3.connect("local_runtime_state.db",check_same_thread=False))

# ---- 5. Compile Graph ----
graph = builder.compile(checkpointer=checkpointer)

# ---- 6. Execute Graph (Local Runtime Execution) ----
result = graph.invoke(
    {"count": 0},
    config={
        "recursion_limit": 10,
        "configurable": {"thread_id": "demo-thread-1"}
    }
)
print("\nFinal State:", result)


Increment: 0
Increment: 1
Increment: 2
Increment: 3
Increment: 4

Final State: {'count': 5}
